In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("movies.csv")
df2 = df1.drop("genres",axis = 1)
df2.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [3]:
df3 = pd.read_csv("ratings.csv")
df4 = df3.drop("timestamp",axis = 1)
df4.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
movie = df2.merge(df4)
movie.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [5]:
def get_top_rated(movie, thr = 80, no = 10):
    rate_mean=movie.groupby('title').rating.mean()
    rate = pd.DataFrame(rate_mean)
    rate["no"] = movie.groupby("title").rating.count()
    df5 = rate[rate["no"]>thr]
    maxno = df5["no"].max()
    df5["rating"] += df5["no"]/maxno - 0.5
    return df5.sort_values(["rating"],ascending = False).head(no)
get_top_rated(movie)

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,rating,no
title,,
"Shawshank Redemption, The (1994)",4.892548,317
Forrest Gump (1994),4.664134,329
Pulp Fiction (1994),4.630199,307
"Matrix, The (1999)",4.537431,278
"Silence of the Lambs, The (1991)",4.509315,279
Star Wars: Episode IV - A New Hope (1977),4.493994,251
Fight Club (1999),4.435550,218
Schindler's List (1993),4.393693,220
"Godfather, The (1972)",4.372649,192


In [6]:
movie_matrix = movie.pivot_table(index = "userId", columns = "title", values = "rating")
movie_matrix.head(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
movie_matrix.shape

(610, 9719)

In [8]:
def get_sim(movie_matrix, movie_name, thr = 40, no = 5):
    user_rating = movie_matrix[movie_name]
    similar = movie_matrix.corrwith(user_rating)
    corr_matrix = pd.DataFrame(similar, columns = ["correlation"])
    corr_matrix.dropna(inplace = True)
    rate = movie.groupby("title").rating.count()
    rate_temp = pd.DataFrame(rate)
    corr = corr_matrix.merge(rate_temp, on = "title")
    return corr[corr['rating']>thr].sort_values(by = 'correlation', ascending = False).head(10)
get_sim(movie_matrix, "Forrest Gump (1994)", 5)

C:\Users\HP\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\HP\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,correlation,rating
title,,
Eye for an Eye (1996),1.000000,6
Forrest Gump (1994),1.000000,329
Everest (1998),1.000000,6
"Adventures of Pinocchio, The (1996)",1.000000,6
"Other Woman, The (2014)",0.994558,6
Friday the 13th Part IV: The Final Chapter (1984),0.973329,6
"Wave, The (Welle, Die) (2008)",0.973124,7
Kolya (Kolja) (1996),0.968822,7
Quills (2000),0.968496,6


In [9]:
def get_sim_user_reviews(df4, movie_matrix, uid, thr):
    sim_rate = movie_matrix.loc[uid]
    similar_to = movie_matrix.corrwith(sim_rate, axis = 1)
    corr = pd.DataFrame(similar_to, columns=["correlation"])
    corr.dropna(inplace = True)
    coUsers = df4.groupby("userId").rating.count()
    coUsers = pd.DataFrame(coUsers)
    coUsers.columns = ["no"]
    corr = corr.join(coUsers['no'])
    corr = corr[corr['no']>thr]
    corr = corr.sort_values(by='correlation', ascending = False)
    corr = corr[corr['correlation']>0.5]
    corr = corr.sort_values(by='correlation', ascending = False)
    uids = corr.index
    newDf = df4[df4.userId.isin(uids)]
    return newDf
def get_sim_user_top_rated(df4, movie_matrix, uid, thr= 10):
    newDf = get_sim_user_reviews(movie, movie_matrix, uid, thr=10)
    return get_top_rated(newDf, 5, 20)

In [10]:
get_sim_user_top_rated(df4, movie_matrix, 5)

C:\Users\HP\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\HP\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,rating,no
title,,
Pulp Fiction (1994),5.112500,40
Schindler's List (1993),4.749074,27
"Silence of the Lambs, The (1991)",4.703571,35
Casablanca (1942),4.638636,11
"Usual Suspects, The (1995)",4.596739,23
"Matrix, The (1999)",4.590385,39
"Shawshank Redemption, The (1994)",4.582576,33
Jaws (1975),4.558333,9
Forrest Gump (1994),4.552778,36
